In [ ]:
!pip install kaggle


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Petals"

In [ ]:
!kaggle competitions download -c tpu-getting-started

100% 4.77G/4.79G [00:45<00:00, 134MB/s]
100% 4.79G/4.79G [00:45<00:00, 113MB/s]


In [ ]:
!unzip "/content/drive/MyDrive/tpu-getting-started.zip" -d "/content/drive/MyDrive/Petals/"

In [ ]:
from tensorflow.keras.layers import Dense,LSTM,Embedding,Dropout,Bidirectional,GRU,Flatten,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import preprocessing
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
import pandas as pd 
import numpy as np
from sklearn.feature_extraction import FeatureHasher
from tensorflow.keras.metrics import MeanSquaredLogarithmicError
from tensorflow.keras.applications import InceptionV3,ResNet50V2,EfficientNetB2
import os
import tensorflow as tf
from tensorflow import keras

In [ ]:
GCS_DS_PATH = '/content/drive/MyDrive/Colab Notebooks/Petals'

In [ ]:
IMAGE_SIZE = [224, 224]
EPOCHS = 100
BATCH_SIZE = 64
NUM_TRAINING_IMAGES = 12753
NUM_TEST_IMAGES = 7382
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
AUTOTUNE = tf.data.AUTOTUNE

Data augumentation is used , as there a lot of classes for classification and with a small dataset , data agumentation can help produce randomly transformed images of acutal images and are classified to the same label. This helps the model to extract key features more precessively , even when some parts of the images are missing or have different contrast or translated along both the axis or zoomed in etc. It essesntially helps us increase our training dataset. Keras preprocessing layers are used and they are applied to the datasets during the time of training.


In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  layers.RandomTranslation(0.2,0.2,fill_mode="reflect"),
  # layers.RandomContrast(0.3),
  layers.RandomZoom(0.1)
])

Random erasing function is used to randomly erase a certain part of the image , creating a black mask. The function first gets the dimensions of the image and using sh and sl which correspond to max and min ratio of total image area and computes the upper and lower bound of possible erasing area. Then the maximum possible height and width of the erasing area is found. Then the position of the erasing is area is randomly found and finally it is applied onto the image. 

In [ ]:
def random_erasing(img, sl=0.1, sh=0.2, rl=0.4, p=0.3):
    h = tf.shape(img)[0]
    w = tf.shape(img)[1]
    c = tf.shape(img)[2]
    origin_area = tf.cast(h*w, tf.float32)

    e_size_l = tf.cast(tf.round(tf.sqrt(origin_area * sl * rl)), tf.int32)
    e_size_h = tf.cast(tf.round(tf.sqrt(origin_area * sh / rl)), tf.int32)

    e_height_h = tf.minimum(e_size_h, h)
    e_width_h = tf.minimum(e_size_h, w)

    erase_height = tf.random.uniform(shape=[], minval=e_size_l, maxval=e_height_h, dtype=tf.int32)
    erase_width = tf.random.uniform(shape=[], minval=e_size_l, maxval=e_width_h, dtype=tf.int32)

    erase_area = tf.zeros(shape=[erase_height, erase_width, c])
    erase_area = tf.cast(erase_area, tf.uint8)

    pad_h = h - erase_height
    pad_top = tf.random.uniform(shape=[], minval=0, maxval=pad_h, dtype=tf.int32)
    pad_bottom = pad_h - pad_top

    pad_w = w - erase_width
    pad_left = tf.random.uniform(shape=[], minval=0, maxval=pad_w, dtype=tf.int32)
    pad_right = pad_w - pad_left

    erase_mask = tf.pad([erase_area], [[0,0],[pad_top, pad_bottom], [pad_left, pad_right], [0,0]], constant_values=1)
    erase_mask = tf.squeeze(erase_mask, axis=0)
    erased_img = tf.multiply(tf.cast(img,tf.float32), tf.cast(erase_mask, tf.float32))

    return tf.cond(tf.random.uniform([], 0, 1) > p, lambda: tf.cast(img, img.dtype), lambda:  tf.cast(erased_img, img.dtype))

This function is used to decode the image from the tensor and convert the pixel values between 0,1.

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image


This functions maps each of the image obtained from the tfrecord , which contains the image and label as tensors to their respective types and decode image is called to decode the image.

In [ ]:
def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

This function is used to load the datasets from the tfrecords files , based on labeled value ie , whether it is train or test data the appropriate function for mapping the records is called .

In [ ]:
def load_dataset(filenames, labeled=True, ordered=False):

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False

    dataset = tf.data.TFRecordDataset(filenames) 
    dataset = dataset.with_options(ignore_order) 
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord,num_parallel_calls = AUTOTUNE)


    return dataset

The following functions load the files from the local directory and get the tfrecords mapped to the corresponding types using the above functions. Augumentation and erasing functions are applied to the function using the map function of tf datasets. Repeat is used as the dataset has to fetch the values for every epoch , and shuffle enables shuffling the data in the buffer. Prefetch is called so when training of one epoch is going on simultaneously the elements for next epoch or fetched to the buffer , the buffer size is dynamically chosen using Autotune depending upon the availablility of resources.

In [ ]:
def get_training_dataset():
    dataset = load_dataset(tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-224x224/train/*.tfrec'), labeled=True)
    dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=AUTOTUNE)
    dataset = dataset.map(lambda x, y: (random_erasing(x), y), 
                num_parallel_calls=AUTOTUNE)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
  
    return dataset.prefetch(buffer_size=AUTOTUNE)

def get_validation_dataset():
    dataset = load_dataset(tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-224x224/val/*.tfrec'), labeled=True, ordered=False)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    return dataset
    

Getting our training and validation Datasets.

In [ ]:
training_dataset = get_training_dataset()
validation_dataset = get_validation_dataset()

Inception V3 is used with the weights of imagenet , include top is set to false because we will provide our own head to our model as the classification is different when comapred to imagenet. The first 249 layers are not trained. An another way would be training our head by freezing the whole inception model and then fine tuning it  by unfreezing a few layers . It can also be trained using a custom training loop if required. But for simplicity im training few of the higher layers of the inception model as a part of the main train run.

In [ ]:
inception = InceptionV3(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])

for layer in inception.layers[:249]:
   layer.trainable = False
for layer in inception.layers[249:]:
   layer.trainable = True

87910968/87910968 [==============================] - 2s 0us/step


Our Model is created , again for simplicity purpose im using the sequntial model and buling feed forward network as our head. Loss is sparse_categorical_crossentropy as we are doing multi-label classification. 

In [ ]:
model = Sequential()
model.add(inception)
model.add(layers.Flatten())
model.add(Dense(512,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(Dense(256,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(Dense(104,activation='softmax'))
model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'])

The appropriate callbacks are set 

In [ ]:
callbacks_list = [
tf.keras.callbacks.ModelCheckpoint(
    filepath='/tmp/checkpoint',
    save_weights_only=True,
    monitor='val_sparse_categorical_accuracy',
    mode='max',
    save_best_only=True),
keras.callbacks.ReduceLROnPlateau(
monitor='sparse_categorical_accuracy',
factor=0.1,
patience=3,
)
]

Finally fitting our model 

In [ ]:
historical = model.fit(training_dataset, 
          steps_per_epoch=STEPS_PER_EPOCH, 
          epochs=EPOCHS, 
          validation_data=validation_dataset,callbacks=callbacks_list)

In [ ]:
resnet = ResNet50V2(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3]) 


94668760/94668760 [==============================] - 1s 0us/step


In [ ]:
for layer in resnet.layers[:80]:
   layer.trainable = False
for layer in resnet.layers[81:]:
   layer.trainable = True

In [ ]:
model2 = Sequential()
model2.add(resnet)
model2.add(layers.Flatten())
model2.add(Dense(512,activation='relu'))
model2.add(layers.Dropout(0.5))
model2.add(Dense(256,activation='relu'))
model2.add(layers.Dropout(0.5))
model2.add(Dense(104,activation='softmax'))
model2.compile(optimizer=optimizers.Adam(learning_rate=0.001),
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'])

In [ ]:
model2_history = model2.fit(training_dataset, 
          steps_per_epoch=STEPS_PER_EPOCH, 
          epochs=EPOCHS,callbacks=callbacks_list, 
          validation_data=validation_dataset)